In [1]:
#using Pkg; Pkg.add("BenchmarkTools"); Pkg.add("StatsBase")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


In [2]:
using Random
mutable struct Xorshift <: AbstractRNG
    state::UInt64
end
Xorshift() = Xorshift(rand(RandomDevice(), UInt64))

Xorshift

In [3]:
Random.seed!(r::Xorshift, seed::UInt64 = rand(RandomDevice(), UInt64)) = r.state = seed

function xorshift_rand(r::Xorshift)
    state = r.state
    state ⊻= state << 13
    state ⊻= state >> 7
    state ⊻= state << 17
    r.state = state
end

xorshift_rand (generic function with 1 method)

In [4]:
const XorshiftSamplers = Union{map(T->Random.SamplerType{T},
        [Bool, UInt32, Int32, UInt64, Int64])...}

Union{Random.SamplerType{Bool}, Random.SamplerType{Int32}, Random.SamplerType{Int64}, Random.SamplerType{UInt32}, Random.SamplerType{UInt64}}

In [5]:
Base.rand(r::Xorshift, sampler::XorshiftSamplers) = xorshift_rand(r) % sampler[]

In [6]:
Random.rng_native_52(::Xorshift) = UInt64

In [7]:
using StatsBase
r = Xorshift(0x0139408dcbbf7a44)

Xorshift(0x0139408dcbbf7a44)

In [8]:
countmap(rand(r, 1:10, 10^8))

Dict{Int64,Int64} with 10 entries:
  7  => 9996777
  4  => 9997119
  9  => 10004173
  10 => 9998711
  2  => 10000084
  3  => 9998660
  5  => 10002696
  8  => 10001941
  6  => 9996874
  1  => 10002965

In [9]:
const X = zeros(Int, 5, 5)

5×5 Array{Int64,2}:
 0  0  0  0  0
 0  0  0  0  0
 0  0  0  0  0
 0  0  0  0  0
 0  0  0  0  0

In [10]:
foreach(i -> X[rand(r, 1:5), rand(r, 1:5)] += 1, 1:10^7)

In [11]:
25 * X / 10^7

5×5 Array{Float64,2}:
 0.99747   1.00078   0.999343  1.00031   0.99737 
 0.996387  1.00425   1.00035   0.999225  1.00246 
 0.99828   1.00058   1.00111   1.00032   0.99703 
 1.0003    0.999223  1.00174   1.00182   0.999058
 1.00076   0.99959   1.00011   0.999045  1.00309 

In [12]:
rand(Xorshift(), UInt128)


ArgumentError: ArgumentError: Sampler for this object is not defined

In [13]:
function Base.rand(r::Xorshift, sampler::Random.SamplerType{UInt128})
    r1 = rand(r, UInt64)
    r2 = rand(r, UInt64)
    (UInt128(r1) << 64) | r2
end

In [14]:
rand(Xorshift(), UInt128)

0x0586c91d79ac483f84d56e715f94ff6f

In [15]:
using BenchmarkTools

In [16]:
r = Xorshift()

Xorshift(0xc4589f1aaef27ceb)

In [17]:
@benchmark rand()

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.905 ns (0.00% GC)
  median time:      8.709 ns (0.00% GC)
  mean time:        9.483 ns (0.00% GC)
  maximum time:     187.087 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [18]:
@benchmark rand($r)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     5.005 ns (0.00% GC)
  median time:      8.108 ns (0.00% GC)
  mean time:        8.725 ns (0.00% GC)
  maximum time:     179.880 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999